### Needed libraries

In [ ]:
import rawpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration
RAW_DIR = Path('./')
OUTPUT_EXCEL = 'mean_rgb_values_center.xlsx'
IMAGE_RANGE = range(360, 751, 10)
CROP_SIZE = 500  # half-width/height of crop region
NORMALIZE_DIVISOR = 1000  # scale RGB values

# Initialize DataFrame
rgb_data = pd.DataFrame(columns=['Image', 'Red', 'Green', 'Blue'])


def process_raw_image(file_path: Path, crop_size: int = 500) -> tuple:
    """
    Process a RAW image and return min-max normalized RGB values from the center crop,
    such that each value is scaled between 0 and 1 based on the min and max of the RGB means.
    """
    try:
        with rawpy.imread(str(file_path)) as raw:
            rgb = raw.postprocess(gamma=(1, 1), no_auto_bright=True, output_bps=8)
            h, w = rgb.shape[:2]
            crop = rgb[h//2 - crop_size:h//2 + crop_size, w//2 - crop_size:w//2 + crop_size]
            mean_rgb = np.mean(crop, axis=(0, 1)).astype(np.float32)

            min_val = np.min(mean_rgb)
            max_val = np.max(mean_rgb)

            if max_val - min_val == 0:
                return 0.0, 0.0, 0.0  # Avoid division by zero when all channels are equal

            normalized_rgb = (mean_rgb - min_val) / (max_val - min_val)
            return tuple(normalized_rgb)
    except Exception as e:
        print(f"[ERROR] Failed to process {file_path.name}: {e}")
        return None, None, None



def collect_rgb_data():
    """
    Process all RAW images in the specified range and collect center RGB values.
    """
    for wavelength in IMAGE_RANGE:
        file_path = RAW_DIR / f'{wavelength}.NEF'
        if file_path.exists():
            r, g, b = process_raw_image(file_path, crop_size=CROP_SIZE, normalize=NORMALIZE_DIVISOR)
            if r is not None:
                rgb_data.loc[len(rgb_data)] = {'Image': wavelength, 'Red': r, 'Green': g, 'Blue': b}
        else:
            print(f"[WARNING] File not found: {file_path}")

def save_data_to_excel(df: pd.DataFrame, file_name: str):
    """
    Save RGB data to Excel.
    """
    df.to_excel(file_name, index=False)
    print(f"[INFO] Saved mean RGB values to {file_name}")

def plot_rgb_sensitivity(df: pd.DataFrame):
    """
    Plot the RGB spectral sensitivity curve.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(df.Image, df.Red, color='red', lw=2, alpha=0.7, label='Red Channel')
    plt.plot(df.Image, df.Green, color='green', lw=2, alpha=0.7, label='Green Channel')
    plt.plot(df.Image, df.Blue, color='blue', lw=2, alpha=0.7, label='Blue Channel')

    plt.title('Recovered Nikon D850 Spectral Sensitivity')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Relative Intensity')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Main execution
if __name__ == '__main__':
    collect_rgb_data()
    save_data_to_excel(rgb_data, OUTPUT_EXCEL)

    # Reload and plot
    df = pd.read_excel(OUTPUT_EXCEL)
    plot_rgb_sensitivity(df)
